In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, KNNImputer
import numpy as np
from pathlib import Path

folder_path = "./data/"

In [12]:
def get_df_and_truth(data_list, truth_list, cols=['Chr', 'START_POS_REF', 'END_POS_REF'], split=False, use_small=True):
    
    df_res = []
    truth_res = {}

    extra = "" if use_small else "-all"
    for data_name, truth_name in zip(data_list, truth_list):
        data_path = folder_path + f"snv-parse-{data_name}{extra}.txt"
        df = pd.read_csv(data_path, sep='\t', low_memory=False)
        df['is_real'] = data_name[:4] == 'real'
        df['ds_name'] = data_name
        df['ds_type'] = data_name[:5]
        truth_path = folder_path + f"{data_name}/{truth_name}.bed"
        truth = pd.read_csv(truth_path, sep='\t', header=None, names = ['Chr', 'START_POS_REF', 'END_POS_REF'])
        truth['label'] = 1
        df_res.append(df)
        truth_res[data_name] = truth
    
    return df_res, truth_res

def get_df(data_name):
    
    data_path = folder_path + f"snv-parse-{data_name}.txt"
    df = pd.read_csv(data_path, sep='\t', low_memory=False)
    df['is_real'] = data_name[:4] == 'real'
    df['ds_name'] = data_name
    df['ds_type'] = data_name[:5]
    df['label'] = -1
    
    return df

In [13]:
data_list = ["real1", "real2_part1", "syn1", "syn2", "syn3", "syn4", "syn5"]
#data_list = ["real1", "real2_part1"]
truth_list = ["real1_truth", "real2_truth_chr1to5", "syn1_truth", "syn2_truth", "syn3_truth", "syn4_truth", "syn5_truth" ]
#truth_list = ["real1_truth", "real2_truth_chr1to5"]
df_list, truth_dict = get_df_and_truth(data_list, truth_list, use_small=True)

In [29]:
merged_list = [df.merge(truth, how='left').fillna({'label':0}) for df, truth in zip(df_list, truth_dict.values())] + [get_df("real2_part2")]
merged_list = [df.drop(labels=['vd_SAMPLE', 'vd_STATUS', 'vd_TYPE'], axis=1) for df in merged_list]
merged_list[0].apply(lambda x: len(x.unique()))

Chr                      73
START_POS_REF         49294
END_POS_REF           49294
REF                       4
ALT                       4
REF_MFVdVs               60
ALT_MFVdVs              590
Sample_Name               1
FILTER_Mutect2            2
FILTER_Freebayes          2
FILTER_Vardict            2
FILTER_Varscan            2
m2_ClippingRankSum        5
m2_DP                   923
m2_ECNT                  34
m2_FS                  4856
m2_MQ                  2086
m2_MQ0                  354
m2_MQRankSum           3763
m2_ReadPosRankSum      4408
f_AN                      4
f_DP                    594
f_DPB                   594
f_EPPR                 2110
f_GTI                     4
f_MQMR                 4412
f_NS                      2
f_NUMALT                  3
f_ODDS                 6402
f_PAIREDR              1313
f_PQR                     2
f_PRO                     2
f_QR                   3745
f_RO                    526
f_RPPR                 2273
f_SRF               

In [34]:
def handle_inf_values(df, inf_list=["vd_SOR"]):
    # for col in inf_list:
    #    df.loc[df.loc[:,col] == float('inf'), col] = -1
    df = df.drop(labels=inf_list, axis=1)
    return df

def handle_filters(df, columns):
    res_df = [df]
    res_cols = []

    for col in columns:
        cat_filter_df = pd.get_dummies(df.loc[:,col].str.split(';').explode()).groupby(level=0).sum()
        cat_filter_df = cat_filter_df.rename(lambda x: f'{col}_{x}', axis=1)
        res_df.append(cat_filter_df)
        res_cols += list(cat_filter_df.columns)

    return pd.concat(res_df, axis=1), res_cols
    
def handle_nan_values(d_list, sample_list, imputer):
    
    df = pd.concat(d_list, axis=0).reset_index(drop=True)

    #filter_list = ["FILTER_Mutect2", "FILTER_Freebayes","FILTER_Vardict","FILTER_Varscan"]
    #df, filter_cols = handle_filters(df, filter_list[1:])
    df = handle_inf_values(df)
    
    #nan_cols = list(set(df.columns[df.isna().any()]) - set(filter_list))
    nan_cols = list(set(df.columns[df.isna().any()]))
    new_nan_cols = [x + "_nan_ind" for x in nan_cols]
    df[new_nan_cols] = df.loc[:,nan_cols].isna()
    
    df.loc[df['ds_name'].isin(sample_list),nan_cols] = imputer.fit_transform(df.loc[df['ds_name'].isin(sample_list),nan_cols])
    if not df['ds_name'].isin(sample_list).all():
        df.loc[~df['ds_name'].isin(sample_list),nan_cols] = imputer.transform(df.loc[~df['ds_name'].isin(sample_list),nan_cols])
        
    return df, nan_cols, new_nan_cols
    
def create_more_cat_values(df, columns):
    
    ret_cols = []
    for c in columns:
        s = df[c].str.split('/').apply(lambda x: x[:-1])
        new_cols = [c + f"_{i}" for i in range(len(s.iloc[0]))]
        df[new_cols] = pd.DataFrame(np.vstack(s.values))
        ret_cols += new_cols

        for col in new_cols:
            rare_cols = df.loc[:,col].value_counts()[(df.loc[:,col].value_counts() <= 200)].index
            df.loc[df.loc[:,col].isin(rare_cols),col] = 'rare'
            
    return df, ret_cols
        
def handle_cat_values(df, cat_list):
    cat_df = pd.get_dummies(df, columns=cat_list, drop_first=True)
    cat_cols = list(set(cat_df.columns) - set(df.columns))
    cat_df['Chr'] = df['Chr']
    return cat_df, cat_cols

def get_X_y(df, sample_list, feature_list):
    df = df[df['ds_name'].isin(sample_list)]
    X = df.loc[:,feature_list].astype(float)
    y = df.loc[:,"label"].astype(float)
    return X, y

def preprocess_data(train_sample_list):
    col_list = ["is_real"]

    # Handling nan values
    #imputer=SimpleImputer(strategy='constant', fill_value=0)
    imputer=SimpleImputer(strategy='median')
    #imputer=KNNImputer(n_neighbors=10)
    df, nan_cols, new_nan_cols = handle_nan_values(merged_list, train_sample_list, imputer)
    
    # Handling Categorical Data
    ret_cols = []
    add_cols = ["REF_MFVdVs", "ALT_MFVdVs"]
    df, ret_cols = create_more_cat_values(df, add_cols)
    
    cat_cols = []#, "ALT"]
    cat_list = ['ds_type', "REF", "ALT"] + ret_cols
    df, cat_cols = handle_cat_values(df, cat_list)
    
    # Combining all features and getting training data
    feature_list = col_list + nan_cols + new_nan_cols + cat_cols
    return df, feature_list

In [35]:
train_sample_list = ['real1']
df, feature_list = preprocess_data(train_sample_list)

In [36]:
from sklearn.utils.class_weight import compute_class_weight

def calc_F1(pred, truth):
    predv = pred['Chr'].astype(str) + pred['START_POS_REF'].astype(str)
    truthv = truth['Chr'].astype(str) + truth['START_POS_REF'].astype(str)

    res = pd.DataFrame(columns=['TP', 'FP', 'FN', 'Precision', 'Recall', 'F1'])

    res.loc[0, 'TP'] = sum(predv.isin(truthv))
    res.loc[0, 'FP'] = sum(~predv.isin(truthv))
    res.loc[0, 'FN'] = sum(~truthv.isin(predv))

    res.loc[0, 'Precision'] = res.loc[0, 'TP'] / (res.loc[0, 'TP'] + res.loc[0, 'FP'])
    res.loc[0, 'Recall'] = res.loc[0, 'TP'] / (res.loc[0, 'TP'] + res.loc[0, 'FN'])
    res.loc[0, 'F1'] = (2 * res.loc[0, 'Precision'] * res.loc[0, 'Recall']) / (res.loc[0, 'Precision'] + res.loc[0, 'Recall'])

    return res

def get_class_weights(y):    
    classes = np.unique(y)
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
    class_weights = dict(zip(classes, weights))
    return class_weights

def get_pred(train_ds_name, test_ds_name, model_class, model_params, threshold=0.5, syn_sample_weight=0.001):
    X_train, y_train = get_X_y(df, train_ds_name, feature_list)

    #class_weight = get_class_weights(y_train.loc[X_train['is_real'].astype(bool)])
    class_weight = get_class_weights(y_train.loc[X_train['is_real'].astype(bool)])
    model = model_class.set_params(**model_params, class_weight=class_weight)
    sample_weight = X_train['is_real'] + syn_sample_weight * (1 - X_train['is_real'])
    model.fit(X_train, y_train, sample_weight)
    
    X_test, y_test = get_X_y(df, [test_ds_name], feature_list)
    pred_proba = model.predict_proba(X_test)
    y_pred = (pred_proba[:,1] >= threshold)  

    return y_pred
    
def run_test(train_ds_name, test_ds_name, model_class, model_params, threshold=0.5, syn_sample_weight=0.001):

    y_pred = get_pred(train_ds_name, test_ds_name, model_class, model_params, threshold, syn_sample_weight)
    pred = df[df["ds_name"]==test_ds_name].loc[y_pred.astype(bool),['Chr', 'START_POS_REF', 'END_POS_REF']]
    res = calc_F1(pred, truth_dict[test_ds_name])
    
    return res

def save_pred(train_ds_name, test_ds_name_list, model, model_params, threshold=0.5, syn_sample_weight=0.001):

    Path("./predictions").mkdir(parents=True, exist_ok=True)
    
    X_train, y_train = get_X_y(df, train_ds_name, feature_list)
    class_weight='balanced_subsample'
    #class_weight = get_class_weights(y_train.loc[X_train['is_real'].astype(bool)])
    sample_weight = X_train['is_real'] + syn_sample_weight * (1 - X_train['is_real'])
    model = model.set_params(**model_params, class_weight=class_weight)
    model.fit(X_train, y_train, sample_weight=sample_weight)

    res = []
    for test_ds_name in test_ds_name_list:
        X_test, y_test = get_X_y(df, [test_ds_name], feature_list)
        y_pred = model.predict(X_test)
        pred = df[df["ds_name"]==test_ds_name].loc[y_pred.astype(bool),['Chr', 'START_POS_REF', 'END_POS_REF']]
        pred.to_csv(f'./predictions/my-{test_ds_name}-predictions.bed', index=False, sep='\t', header=False)
    
        if test_ds_name in truth_dict:
            score = calc_F1(pred, truth_dict[test_ds_name])
            score['ds_name'] = test_ds_name
            res.append(score)
    
    return pd.concat(res)

In [37]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [39]:
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier, StackingClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42, max_depth=25, verbose=2)
gb = HistGradientBoostingClassifier()
lgbm = LGBMClassifier(n_estimators=1000, learning_rate=0.05,verbose=2, n_jobs=-1, random_state=42)
lgr = LogisticRegression(C=1000)
extra = ExtraTreesClassifier(n_estimators=1000, n_jobs=-1, random_state=42, verbose=2)
extra_params = {"n_estimators" : 1000, "min_samples_split" : 10, "max_depth" : 48, "verbose" : 1, "n_jobs" : -1, "random_state" : 42}
catboost = CatBoostClassifier()
catboost_params = {"num_trees" : 1000, "learning_rate" : 0.09, "depth" : 5, "task_type" : "GPU"}
xgb = XGBClassifier()
xgb_params = {"n_estimators" : 1000, "learning_rate": 0.1, "random_state" : 42}

In [42]:
from copy import copy
rf = RandomForestClassifier()
rf_params = {"n_estimators" : 100, "min_samples_split" : 2, "max_depth" : 64, "oob_score" : False, "min_samples_leaf" : 1, "bootstrap" : True, "verbose" : 1, "n_jobs" : -1, "random_state" : 42}

#model, model_params = extra, extra_params
model, model_params = rf, rf_params
#model, model_params = copy(catboost), copy(catboost_params)
#model, model_params = xgb, xgb_params

In [43]:
results = save_pred(['real1', 'syn1', 'syn2', 'syn3', 'syn4', 'syn5'], ["real1", "real2_part1"], model, model_params, syn_sample_weight=0.1)
results.iloc[0]['F1'], results.iloc[1]['F1']

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    7.4s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    0.0s finished


(0.9826455842653298, 0.8554913294797687)

In [51]:
model, model_params = rf, rf_params
rf_results = save_pred(['real1', 'syn1', 'syn2', 'syn3', 'syn4', 'syn5'], ["real2_part1"], model, model_params, syn_sample_weight=0.1)
model, model_params = xgb, xgb_params
#xgb_results = save_pred(['real1', 'syn1', 'syn2', 'syn3', 'syn4', 'syn5'], ["real2_part1"], model, model_params, syn_sample_weight=0.1)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    6.7s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    0.0s finished


In [52]:
results.set_index('ds_name')

,TP,FP,FN,Precision,Recall,F1
ds_name,,,,,,
real2_part1,370,4,121,0.989305,0.753564,0.855491
